## Deliverable 2. Create a Customer Travel Destinations Map.

In [42]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import Google_API_KEY

# Configure gmaps API key
gmaps.configure(api_key=Google_API_KEY)

In [43]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Wind Direction,Weather,Country,Date
0,0,Rikitea,-23.1203,-134.9692,80.28,74,24,9.44,30,few clouds,PF,2022-03-19 05:49:23
1,1,Mar Del Plata,-38.0023,-57.5575,50.88,70,0,5.01,271,clear sky,AR,2022-03-19 05:49:24
2,2,Puerto Ayora,-0.7393,-90.3518,77.88,80,24,4.81,158,few clouds,EC,2022-03-19 05:46:26
3,3,Port Alfred,-33.5906,26.8910,73.40,85,100,8.48,67,overcast clouds,ZA,2022-03-19 05:49:25
4,4,Kapaa,22.0752,-159.3190,78.78,75,40,14.97,50,scattered clouds,US,2022-03-19 05:49:25


In [44]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 75


In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Wind Direction,Weather,Country,Date
1,1,Mar Del Plata,-38.0023,-57.5575,50.88,70,0,5.01,271,clear sky,AR,2022-03-19 05:49:24
3,3,Port Alfred,-33.5906,26.8910,73.40,85,100,8.48,67,overcast clouds,ZA,2022-03-19 05:49:25
5,5,Westport,41.1415,-73.3579,53.85,96,100,5.01,161,fog,US,2022-03-19 05:49:25
9,9,Rio Grande,-32.0350,-52.0986,64.51,83,100,19.44,197,overcast clouds,BR,2022-03-19 05:49:27
11,11,Albany,42.6001,-73.9662,50.95,79,100,1.54,102,overcast clouds,US,2022-03-19 05:49:28
13,13,Margate,51.3813,1.3862,47.30,75,0,5.01,92,clear sky,GB,2022-03-19 05:49:29
16,16,Cape Town,-33.9258,18.4232,67.10,76,0,11.50,230,clear sky,ZA,2022-03-19 05:46:25
17,17,Hermanus,-34.4187,19.2345,69.80,85,12,10.36,89,few clouds,ZA,2022-03-19 05:49:30
18,18,Comodoro Rivadavia,-45.8667,-67.5000,49.86,66,40,13.80,150,scattered clouds,AR,2022-03-19 05:49:30
19,19,Korla,41.7597,86.1469,55.11,37,100,13.98,52,overcast clouds,CN,2022-03-19 05:49:31


In [46]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.notnull().sum()


City_ID           270
City              270
Lat               270
Lng               270
Max Temp          270
Humidity          270
Cloudiness        270
Wind Speed        270
Wind Direction    270
Weather           270
Country           265
Date              270
dtype: int64

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [48]:
preferred_cities_df.notnull().sum()

City_ID           265
City              265
Lat               265
Lng               265
Max Temp          265
Humidity          265
Cloudiness        265
Wind Speed        265
Wind Direction    265
Weather           265
Country           265
Date              265
dtype: int64

In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,50.88,clear sky,-38.0023,-57.5575,
3,Port Alfred,ZA,73.40,overcast clouds,-33.5906,26.8910,
5,Westport,US,53.85,fog,41.1415,-73.3579,
9,Rio Grande,BR,64.51,overcast clouds,-32.0350,-52.0986,
11,Albany,US,50.95,overcast clouds,42.6001,-73.9662,
13,Margate,GB,47.30,clear sky,51.3813,1.3862,
16,Cape Town,ZA,67.10,clear sky,-33.9258,18.4232,
17,Hermanus,ZA,69.80,few clouds,-34.4187,19.2345,
18,Comodoro Rivadavia,AR,49.86,scattered clouds,-45.8667,-67.5000,
19,Korla,CN,55.11,overcast clouds,41.7597,86.1469,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")   
        

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


In [51]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.notnull().sum()

City          265
Country       265
Max Temp      265
Weather       265
Lat           265
Lng           265
Hotel Name    265
dtype: int64

In [52]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [53]:
hotel_df

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Mar Del Plata,AR,50.88,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
3,Port Alfred,ZA,73.40,overcast clouds,-33.5906,26.8910,The Halyards Hotel
5,Westport,US,53.85,fog,41.1415,-73.3579,The Inn at Longshore
9,Rio Grande,BR,64.51,overcast clouds,-32.0350,-52.0986,Hotel Atlântico Rio Grande
11,Albany,US,50.95,overcast clouds,42.6001,-73.9662,
...,...,...,...,...,...,...,...
698,South Lake Tahoe,US,46.26,clear sky,38.9332,-119.9844,Tahoe Lakeshore Lodge & Spa
700,Kaifeng,CN,48.45,overcast clouds,34.7911,114.3483,Pullman Kaifeng Jianye
701,Valenca,BR,70.86,scattered clouds,-13.3703,-39.0731,Hotel Galeão
706,Madaoua,NE,70.45,clear sky,14.0730,5.9600,


In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt style="font-weight: bold;">Hotel Name</dt><dd>{Hotel Name}</dd>
<dt style="font-weight: bold;">City</dt><dd>{City}</dd>
<dt style="font-weight: bold;">Country</dt><dd>{Country}</dd>
<dt style="font-weight: bold;">Current Weather</dt><dd>{Weather}</dd>
<dt style="font-weight: bold;">Current Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure


In [56]:
fig

Figure(layout=FigureLayout(height='420px'))